In [1]:
import os

os.chdir("../../")


In [ ]:
from pyspark.sql import SparkSession


spark = (
    SparkSession.builder
    .appName("bronze-preparation")
    .getOrCreate()
)


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/27 16:27:45 WARN Utils: Your hostname, Air-M4.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.65 instead (on interface en0)
25/09/27 16:27:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/27 16:27:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import os

FILES_PATH = f"{os.path.abspath('.')}/data/raw"

ARREST_PATH = os.path.join(FILES_PATH, "arrests")
CRIME_PATH = os.path.join(FILES_PATH, "crimes")
CODE_PATH = os.path.join(FILES_PATH, "iucr")

In [4]:
def listdir_abs(dir_path: str) -> list[str]:
    return [os.path.join(dir_path, filename) for filename in os.listdir(dir_path)]

In [8]:
crimes_df = spark.read.options(header="true", inferSchema="true").csv(path=listdir_abs(CRIME_PATH))
arrest_df = spark.read.options(header="true").csv(path=listdir_abs(ARREST_PATH))
code_df = spark.read.options(header="true").csv(path=listdir_abs(CODE_PATH))

In [9]:
code_df.head()

Row(IUCR='110', PRIMARY DESCRIPTION='HOMICIDE', SECONDARY DESCRIPTION='FIRST DEGREE MURDER', INDEX CODE='I', ACTIVE='true')

In [10]:
code_df.show(5)

+----+--------------------+---------------------+----------+------+
|IUCR| PRIMARY DESCRIPTION|SECONDARY DESCRIPTION|INDEX CODE|ACTIVE|
+----+--------------------+---------------------+----------+------+
| 110|            HOMICIDE|  FIRST DEGREE MURDER|         I|  true|
| 130|            HOMICIDE| SECOND DEGREE MURDER|         I|  true|
| 141|            HOMICIDE| INVOLUNTARY MANSL...|         N|  true|
| 142|            HOMICIDE|    RECKLESS HOMICIDE|         N|  true|
| 261|CRIMINAL SEXUAL A...| AGGRAVATED - HANDGUN|         I|  true|
+----+--------------------+---------------------+----------+------+
only showing top 5 rows


In [11]:
from pyspark.sql.functions import current_timestamp, input_file_name

timestamp_column = {
    "load_timestamp": current_timestamp(),
}

file_name_column = {
    "source_file": input_file_name(),
}



crimes_bronze_df = crimes_df.withColumns({**timestamp_column, **file_name_column})
arrest_bronze_df = arrest_df.withColumns({**timestamp_column, **file_name_column})
code_bronze_df = code_df.withColumns(timestamp_column)

In [17]:
crimes_bronze_df.repartition("source_file").write.format("parquet").mode("append").save("./data/bronze/crimes")
arrest_bronze_df.repartition("source_file").write.format("parquet").mode("append").save("./data/bronze/arrests")
code_bronze_df.write.format("parquet").mode("overwrite").save("./data/bronze/iucr")


25/09/27 16:52:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
